In [1]:
import numpy as np
import os, sys
from scipy import io

import torch
import torch.nn as nn
import torch.optim as optim

sys.path.append(os.path.join(os.path.dirname(""), "..", "src"))
from baseline_model import BaselineCNN, NMRDataset, train_model, evaluate_model
from advanced_model import AdvancedNMRNet, train_advanced_model, evaluate_advanced_model
from model_comparison import run_complete_comparison
from torch.utils.data import DataLoader, random_split

%load_ext autoreload
%autoreload 2


In [2]:
RESULTS_PATH = os.path.join(os.path.dirname(""), "..", "results")
DATA_PATH = os.path.join(os.path.dirname(""), "..", "data")
data_path = os.path.join(DATA_PATH, "TrainingData_vol2")

In [3]:
data_len = 42 # number of different speed / displacement combinations
sample_len = 100 # number of different samples within each combination

In [4]:
mat_contents = {}
for idx in range(data_len):
    # load .mat file
    file_idx = f"0{idx+1}" if idx < 9 else idx+1
    filename = os.path.join(data_path, f"case0{file_idx}_result.mat")
    mat_contents[idx] = io.loadmat(filename)

Prepare echo_shape data for training<br>
Final input shape should be: 54, 12, 4200

In [5]:
all_echo_shape = []
all_mqi = []
all_v = []
all_d = []
for idx_1 in range(data_len):
    for idx_2 in range(sample_len):
        mix_sample = mat_contents[idx_1]['mixed'][0][idx_2]
        all_v.append(mat_contents[idx_1]['v'][0][0])
        all_d.append(mat_contents[idx_1]['d'][0][0])
        all_echo_shape.append(mix_sample[1])
        all_mqi.append(mix_sample[2][0][0])

In [6]:
input_data = np.array(all_echo_shape).reshape(4200, 54, 12)

# expand input data to have real and imaginary numbers
input_data_real = input_data.real
input_data_imag = input_data.imag
input_data_ri = np.array([input_data_real, input_data_imag]).reshape(4200, 2, 54, 12)
print(input_data_ri.shape)

# expand input data to have module and phase
input_data_module = np.abs(input_data)
input_data_phase = np.angle(input_data)
input_data_mp = np.array([input_data_module, input_data_phase]).reshape(4200, 2, 54, 12)
print(input_data_mp.shape)

mqi_values = np.array(all_mqi)

(4200, 2, 54, 12)
(4200, 2, 54, 12)


## Baseline Model

In [7]:
BATCH_SIZE = 32

In [8]:
dataset = NMRDataset(input_data_ri, mqi_values)

In [9]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

In [10]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
# Initialize and train model
model = BaselineCNN(dropout_rate=0.3)
print(f"\nTraining baseline model with {sum(p.numel() for p in model.parameters()):,} parameters")



Training baseline model with 103,777 parameters


In [12]:
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,  # Reduced for quick demo
    learning_rate=0.001
)

Epoch [5/20], Train Loss: 0.1863, Val Loss: 40.1486
Epoch [10/20], Train Loss: 0.1785, Val Loss: 0.2503
Epoch [15/20], Train Loss: 0.1767, Val Loss: 0.1678
Epoch [20/20], Train Loss: 0.1756, Val Loss: 0.1672

Training completed. Best validation loss: 0.1668


In [13]:
# Load best model and evaluate
model.load_state_dict(torch.load(os.path.join(RESULTS_PATH, 'best_baseline_model.pth')))
mse, mae, preds, targets = evaluate_model(model, test_loader)


Test Results:
MSE: 0.1758
MAE: 0.3973
RMSE: 0.4192


## Advanced Model

In [14]:
model = AdvancedNMRNet(dropout_rate=0.3)
print(f"\nTraining advanced model with {sum(p.numel() for p in model.parameters()):,} parameters")


Training advanced model with 2,048,301 parameters


In [15]:
history = train_advanced_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=30,  # Reduced for quick demo
    learning_rate=0.001,
    use_early_stopping=True
)

  → New best model saved (val_loss: 0.1131)
  → New best model saved (val_loss: 0.0920)
  → New best model saved (val_loss: 0.0837)
Epoch [5/30], Train Loss: 0.0868, Val Loss: 0.0835, LR: 0.000655
  → New best model saved (val_loss: 0.0835)
Epoch [10/30], Train Loss: 0.0862, Val Loss: 0.0837, LR: 0.000025
  → New best model saved (val_loss: 0.0834)
Epoch [15/30], Train Loss: 0.0873, Val Loss: 0.0834, LR: 0.000905
Epoch [20/30], Train Loss: 0.0871, Val Loss: 0.0838, LR: 0.000579

Early stopping triggered at epoch 20
Best validation loss: 0.0834
